In [ ]:
from utils import *
import pandas as pd

In [ ]:
orig_file = "/home/pascalgrosset/data/cosmo/vel_analysis/new_hacc/orig-499.haloproperties"
comp_file = "/home/pascalgrosset/data/cosmo/vel_analysis/new_hacc/sz_PosNoComp_vel_.01-499.haloproperties"

In [ ]:
gio_file = pygio.PyGenericIO(orig_file)

# inspect (prints to python stdout, also works in notebook)
gio_file.inspect()

In [ ]:
variables_to_show = ["fof_halo_count"
,"fof_halo_tag"
,"fof_halo_mass"
,"fof_halo_ke"
,"fof_halo_center_x"
,"fof_halo_center_y"
,"fof_halo_center_z"
,"fof_halo_angmom_x"
,"fof_halo_angmom_y"
,"fof_halo_angmom_z"
,"fof_halo_max_cir_vel"
,"fof_halo_com_x"
,"fof_halo_com_y"
,"fof_halo_com_z"
,"fof_halo_com_vx"
,"fof_halo_com_vy"
,"fof_halo_com_vz"
,"fof_halo_1D_vel_disp"]

# Don't exist anymore
#,"fof_halo_mean_vx"
#,"fof_halo_mean_vy"
#,"fof_halo_mean_vz"
#,"fof_halo_vel_disp"

In [ ]:
# Read in the data
data_orig_gio = pygio.read_genericio(orig_file, variables_to_show)
data_comp_gio = pygio.read_genericio(comp_file, variables_to_show)

# Read in the mathched list of halos created from matchHalosPos.py
index_file = read_csv_to_array('/home/pascalgrosset/projects/VizAly-Vis_IO/results/new_Hacc_indices_com_sz_PosNoComp_vel_.01-499.csv',',')

In [ ]:
def adrianVecDiffMag(v1, v2):
    '''Comparison is with Adrian comparison metric:  ||a2 - a1|| / ||a1||: a1=uncompressed, a2=compressed'''
    return ( np.linalg.norm(v2-v1)/ np.linalg.norm(v1) )


def computeMaxDiffMag(indexFile, data_orig_gio, data_comp_gio, numItems):
    '''Create a list of tuples for ang mom diff'''
    data_orig__ang_mom_x = np.array(data_orig_gio["fof_halo_angmom_x"])
    data_orig__ang_mom_y = np.array(data_orig_gio["fof_halo_angmom_y"])
    data_orig__ang_mom_z = np.array(data_orig_gio["fof_halo_angmom_z"])
    
    data_comp__ang_mom_x = np.array(data_comp_gio["fof_halo_angmom_x"])
    data_comp__ang_mom_y = np.array(data_comp_gio["fof_halo_angmom_y"])
    data_comp__ang_mom_z = np.array(data_comp_gio["fof_halo_angmom_z"])
    
    diffMagList = []

    for i in range(numItems):
        index_comp = i
        index_orig = int(indexFile[index_comp])
        
        vec_orig = np.array([data_orig__ang_mom_x[index_orig], data_orig__ang_mom_y[index_orig], data_orig__ang_mom_z[index_orig]])
        vec_comp = np.array([data_comp__ang_mom_x[index_comp], data_comp__ang_mom_y[index_comp], data_comp__ang_mom_z[index_comp]])
    
        magDiff = adrianVecDiffMag(vec_orig, vec_comp)
#         if i == 0:
#             print(vec_orig, vec_comp, magDiff)
#             print(index_orig, index_comp)
        
        diffMagList.append( (i,magDiff) )
        
    return diffMagList



def sortTuple(tup, rev=True): 
    '''Sort a list of tuples in descending order'''
    # reverse = None (Sorts in Ascending order) 
    # key is set to sort using second element of 
    # sublist lambda has been used 
    return(sorted(tup, key = lambda x: x[1], reverse=rev))



def getFilteredList(data, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, orig=True):
    '''Filter the list based on > filter val'''
    fullList_orig=[]
    count = 0
    
    for i in range(num_elem):
        _index_to_show = sortedList[i][0]
        if orig == True:
            __index_to_show = int(index_file[_index_to_show])
        else:
            __index_to_show = _index_to_show
        
        
        if data[filter_var_name][__index_to_show] > filter_var_value:
            #print(data_show_orig_gio[filter_var_name][__index_to_show])
            
            theList = []
            theList.append(sortedList[i][1])
            for var in variables_to_show:
                theList.append(data[var][__index_to_show])

            fullList_orig.append(theList)

            count = count + 1
            if num_to_show != -1:
                if count > num_to_show:
                    break
    
    return fullList_orig

In [ ]:
# Find the halos with the largest difference in ang mom diff
theList = computeMaxDiffMag(index_file, data_orig_gio, data_comp_gio, len(index_file))

# Sort the list in desc order of difference in ang mom diff
sortedList = sortTuple(theList)

In [ ]:
num_to_show = 100
num_elem = len(index_file)

filter_var_name = "fof_halo_count"
filter_var_value = 0


# Use above list to get a list of halos sorted by ang mom diff and filtered by size of halos
_fullList_orig = getFilteredList(data_orig_gio, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, True)
_fullList_comp = getFilteredList(data_comp_gio, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, False)


# Create data frames for easier manipulation
# fof_halo_vel_disappeared
#,"fof_halo_mean_vx"
#,"fof_halo_mean_vy"
#,"fof_halo_mean_vz"
#cols=['mag_diff','fof_halo_count', 'fof_halo_tag', 'fof_halo_mass', 'fof_halo_ke', 'fof_halo_center_x', 'fof_halo_center_y', 'fof_halo_center_z', 'fof_halo_angmom_x', 'fof_halo_angmom_y', 'fof_halo_angmom_z', 'fof_halo_max_cir_vel', 'fof_halo_com_x', 'fof_halo_com_y', 'fof_halo_com_z', 'fof_halo_mean_vx', 'fof_halo_mean_vy', 'fof_halo_mean_vz', 'fof_halo_vel_disp', 'fof_halo_1D_vel_disp']
cols=['mag_diff','fof_halo_count', 'fof_halo_tag', 'fof_halo_mass', 'fof_halo_ke', 'fof_halo_center_x', 'fof_halo_center_y', 'fof_halo_center_z', 'fof_halo_angmom_x', 'fof_halo_angmom_y', 'fof_halo_angmom_z', 'fof_halo_max_cir_vel', 'fof_halo_com_x', 'fof_halo_com_y', 'fof_halo_com_z', 'fof_halo_com_vx', 'fof_halo_com_vy', 'fof_halo_com_vz', 'fof_halo_1D_vel_disp']


_df_orig = pd.DataFrame(_fullList_orig, columns=cols)
_df_comp = pd.DataFrame(_fullList_comp, columns=cols)

_df_orig.to_csv("results/new_hacc_largest_100_ang_mom_diff__orig-499.csv")
_df_comp.to_csv("results/new_hacc_largest_100_ang_mom_diff__PosNoComp_vel_.01-499.csv") 

In [ ]:
_df_orig.head()

In [ ]:
_df_comp.head()

In [ ]:
num_to_show = -1
num_elem = len(index_file)

filter_var_name = "fof_halo_count"
filter_var_value = 10000


# Use above list to get a list of halos sorted by ang mom diff and filtered by size of halos
_fullList_orig = getFilteredList(data_orig_gio, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, True)
_fullList_comp = getFilteredList(data_comp_gio, sortedList, index_file, num_elem, filter_var_name, filter_var_value, num_to_show, False)


# Create data frames for easier manipulation
# Create data frames for easier manipulation
# fof_halo_vel_disappeared
#,"fof_halo_mean_vx"
#,"fof_halo_mean_vy"
#,"fof_halo_mean_vz"
#cols=['mag_diff','fof_halo_count', 'fof_halo_tag', 'fof_halo_mass', 'fof_halo_ke', 'fof_halo_center_x', 'fof_halo_center_y', 'fof_halo_center_z', 'fof_halo_angmom_x', 'fof_halo_angmom_y', 'fof_halo_angmom_z', 'fof_halo_max_cir_vel', 'fof_halo_com_x', 'fof_halo_com_y', 'fof_halo_com_z', 'fof_halo_mean_vx', 'fof_halo_mean_vy', 'fof_halo_mean_vz', 'fof_halo_vel_disp', 'fof_halo_1D_vel_disp']
cols=['mag_diff','fof_halo_count', 'fof_halo_tag', 'fof_halo_mass', 'fof_halo_ke', 'fof_halo_center_x', 'fof_halo_center_y', 'fof_halo_center_z', 'fof_halo_angmom_x', 'fof_halo_angmom_y', 'fof_halo_angmom_z', 'fof_halo_max_cir_vel', 'fof_halo_com_x', 'fof_halo_com_y', 'fof_halo_com_z', 'fof_halo_com_vx', 'fof_halo_com_vy', 'fof_halo_com_vz', 'fof_halo_1D_vel_disp']

_df_orig = pd.DataFrame(_fullList_orig, columns=cols)
_df_comp = pd.DataFrame(_fullList_comp, columns=cols)

_df_orig.to_csv("results/new_hacc_largest_above_10000_ang_mom_diff__orig-499.csv")
_df_comp.to_csv("results/new_hacc_largest_above_10000_ang_mom_diff__PosNoComp_vel_.01-499.csv")

In [ ]:
_df_orig.head()

In [ ]:
_df_comp.head()